In [1]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.5
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [2]:
import tensorflow as tf
print(tf.__version__)

2.7.2


In [3]:
# import the libraries as shown below

from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob
#import matplotlib.pyplot as plt

In [4]:
# re-size all the images to this
IMAGE_SIZE = [224, 224]

train_path = 'C:/Users/priya/Downloads/sp/dataset/train/'
valid_path = 'C:/Users/priya/Downloads/sp/dataset/test/'

In [5]:
# Import the VGG16 library as shown below and add preprocessing layer to the front of VGG
# Here we will be using imagenet weights

vgg16 = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

58900480/58889256 [==============================] - 1s 0us/step


In [6]:
# don't train existing weights
for layer in vgg16.layers:
    layer.trainable = False

In [7]:
folders = glob('C:/Users/priya/Downloads/sp/dataset/test/*')

In [8]:
folders

[]

In [9]:
# our layers - you can add more if you want
x = Flatten()(vgg16.output)

In [10]:
len(folders)

0

In [11]:
prediction = Dense(17, activation='softmax')(x)

# create a model object
model = Model(inputs=vgg16.input, outputs=prediction)

In [12]:
# view the structure of the model
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [13]:
# tell the model what cost and optimization method to use
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [14]:
# Use the Image Data Generator to import the images from the dataset
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen =ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [15]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_cc72ba4862694b39beee7a14fb77f07a = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='XqVhWztQrEwwSEE5mIO-7kLenBRUmJHoIO6z41MfcVp6',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

streaming_body_1 = client_cc72ba4862694b39beee7a14fb77f07a.get_object(Bucket='humannailimageprocessing-donotdelete-pr-vowofyavnadgnv', Key='dataset1.zip')['Body']

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/


In [16]:
from io import BytesIO
import zipfile
unzip = zipfile.ZipFile(BytesIO(streaming_body_1.read()),'r')
file_paths = unzip.namelist()
for path in file_paths:
    unzip.extract(path)

In [17]:
pwd

'/home/wsuser/work'

In [18]:
# Make sure you provide the same target size as initialied for the image size
training_set = train_datagen.flow_from_directory('/home/wsuser/work/dataset/train/',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

Found 647 images belonging to 17 classes.


In [19]:
test_set = test_datagen.flow_from_directory('/home/wsuser/work/dataset/test/',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 183 images belonging to 17 classes.


In [ ]:
# fit the model
r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=20,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

/tmp/wsuser/ipykernel_164/1716041270.py:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  r = model.fit_generator(


Epoch 1/20
21/21 [==============================] - 300s 14s/step - loss: 3.4641 - accuracy: 0.1144 - val_loss: 2.7093 - val_accuracy: 0.1967
Epoch 2/20
21/21 [==============================] - 299s 15s/step - loss: 2.5389 - accuracy: 0.2519 - val_loss: 1.9979 - val_accuracy: 0.3443
Epoch 3/20
21/21 [==============================] - 298s 14s/step - loss: 2.0890 - accuracy: 0.3478 - val_loss: 1.8191 - val_accuracy: 0.4208
Epoch 4/20
21/21 [==============================] - 298s 14s/step - loss: 1.9136 - accuracy: 0.4374 - val_loss: 1.8592 - val_accuracy: 0.5191
Epoch 5/20
21/21 [==============================] - 299s 14s/step - loss: 1.7249 - accuracy: 0.4977 - val_loss: 1.6969 - val_accuracy: 0.5137
Epoch 6/20
21/21 [==============================] - 300s 14s/step - loss: 1.5516 - accuracy: 0.5379 - val_loss: 1.4689 - val_accuracy: 0.5847
Epoch 7/20
21/21 [==============================] - 298s 14s/step - loss: 1.4195 - accuracy: 0.5966 - val_loss: 1.2556 - val_accuracy: 0.6011
Epoch 

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
# plot the loss
plt.plot(r.history['loss'], label='train loss')
plt.plot(r.history['val_loss'], label='val loss')
plt.legend()
plt.show()
plt.savefig('LossVal_loss')

# plot the accuracy
plt.plot(r.history['accuracy'], label='train acc')
plt.plot(r.history['val_accuracy'], label='val acc')
plt.legend()
plt.show()
plt.savefig('AccVal_acc')

In [ ]:
# save it as a h5 file


from tensorflow.keras.models import load_model

model.save('model_vgg16.h5')

In [ ]:
!tar -zcvf human-nail-image-processing-model_new.tgz model_vgg16.h5

In [ ]:
ls -1

In [ ]:
!pip install watson-machine-learning-client --upgrade

In [ ]:
# Replace the credentials that you got from Watson Machine Learning service
from ibm_watson_machine_learning import APIClient
wml_credentials = { 
                    "url": "https://us-south.ml.cloud.ibm.com",
                    "apikey":"sw5lgpdTNhaB-Sa1zX8-Hsarwmnj3WP9huKS8sBQS5Ai"
                  }

client = APIClient(wml_credentials)

In [ ]:
client = APIClient(wml_credentials)

In [ ]:
def guid_from_space_name(client, space_name):
    space = client.spaces.get_details()
    #print(space)
    return(next(item for item in space['resources'] if item['entity']["name"] == space_name)['metadata']['id'])

In [ ]:
space_uid = guid_from_space_name(client, 'humannailimageprocessing')
print("Space UID = " + space_uid)

In [ ]:
client.set.default_space(space_uid)

In [ ]:
client.software_specifications.list()

In [ ]:
software_spec_uid = client.software_specifications.get_uid_by_name("tensorflow_rt22.1-py3.9")
software_spec_uid

In [ ]:
model_details=client.repository.store_model(model='human-nail-image-processing-model_new.tgz',meta_props={
    client.repository.ModelMetaNames.NAME:"CNN",
    client.repository.ModelMetaNames.TYPE:"tensorflow_2.7",
    client.repository.ModelMetaNames.SOFTWARE_SPEC_UID:software_spec_uid}
                                           )
model_id = client.repository.get_model_uid(model_details)
                    

In [ ]:
model_id

In [ ]:
client.repository.download(model_id,'my_model.tar.gz')

In [ ]:
y_pred = model.predict(test_set)

In [ ]:
y_pred

In [ ]:
import numpy as np
y_pred = np.argmax(y_pred, axis=1)

In [ ]:
y_pred

In [ ]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.mobilenet import preprocess_input

In [ ]:
model=load_model('model_vgg16.h5')

In [ ]:
img_data='C:/Users/priya/Downloads/image1.jpg'

In [ ]:
img_data

In [ ]:

streaming_body_3 = client_cc72ba4862694b39beee7a14fb77f07a.get_object(Bucket='humannailimageprocessing-donotdelete-pr-vowofyavnadgnv', Key='image1.jpg')['Body']

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/


In [ ]:
x.shape

In [ ]:
x=x/255

In [ ]:
import numpy as np
import tensorflow as tf
x=np.expand_dims(x,axis=0)
img_data=preprocess_input(x)
img_data.shape

In [ ]:
model.predict(img_data)

In [ ]:
a=np.argmax(model.predict(img_data), axis=1)

In [ ]:
a==1

In [ ]:
import tensorflow as tf

In [ ]:
tf.__version__